## Preparing Data 

#### download frm : [Name Data](https://download.pytorch.org/tutorial/data.zip)

###### Included in the data/names directory are 18 text files named as “[Language].txt”. Each file contains a bunch of names, one name per line, mostly romanized (but we still need to convert from Unicode to ASCII).

######  We’ll end up with a dictionary of lists of names per language, {language: [names ...]}. The generic variables “category” and “line” (for language and name in our case) are used for later extensibility.

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob

data_dir = "./data/names/*.txt"
def findFiles(filePath):
    return glob.glob(filePath)

print("Files \n", findFiles(data_dir), "\n")

import unicodedata, string

all_letters = string.ascii_letters+ " .,;'"
n_letters = len(all_letters)

# Unicode to ascii 
def unicode2ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn'
                  and c in all_letters)

print(unicode2ascii('Ślusàrski'))

# Build category dict, list of names per language 
category_lines = {}
all_categories = []

#Read file and line split 
def readLine(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode2ascii(line) for line in lines]

for fileName in findFiles(data_dir):
    category = fileName.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLine(fileName)
    category_lines[category] = lines

n_categories = len(all_categories)

Files 
 ['./data/names/Arabic.txt', './data/names/Chinese.txt', './data/names/Czech.txt', './data/names/Dutch.txt', './data/names/English.txt', './data/names/French.txt', './data/names/German.txt', './data/names/Greek.txt', './data/names/Irish.txt', './data/names/Italian.txt', './data/names/Japanese.txt', './data/names/Korean.txt', './data/names/Polish.txt', './data/names/Portuguese.txt', './data/names/Russian.txt', './data/names/Scottish.txt', './data/names/Spanish.txt', './data/names/Vietnamese.txt'] 

Slusarski


## Turning Names Into Tensors 
#### Letter -> One hot Vector 
#### Word -> 2d matrix of all One hot Vectors 
Extra dim in pytorch, coz it assumes everything in minibatch. Thus 1 minibatch here



In [ ]:
import tor 

